In [ ]:
import chess
import torch
import numpy as np
from QNet import ChessQNetwork, encode_board  # Import các hàm/class từ code training

# Load model
def load_model(model_path, device='cuda'):
    model = ChessQNetwork()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

# Hàm chơi game
def play_vs_ai(model_path, ai_color=chess.WHITE):
    model = load_model(model_path)
    board = chess.Board()
    
    while not board.is_game_over():
        print("\nCurrent board:")
        print(board)
        
        if board.turn == ai_color:
            # AI move
            move = get_best_move(board, model)
            board.push(move)
            print(f"\nAI moved: {move.uci()}")
        else:
            # Human move
            legal_moves = [move.uci() for move in board.legal_moves]
            while True:
                human_move = input(f"\nYour move ({', '.join(legal_moves)}): ").strip()
                try:
                    board.push_uci(human_move)
                    break
                except:
                    print("Invalid move! Try again.")

    print("\nFinal position:")
    print(board)
    print(f"Game result: {board.result()}")

# Hàm get_best_move (đã sửa đổi)
def get_best_move(board, model, time_limit=3):
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        return None
    
    # Chuẩn bị dữ liệu batch
    state = encode_board(board)
    state_batch = np.tile(state, (len(legal_moves), 1))
    action_features = np.array([[m.from_square/63, m.to_square/63] for m in legal_moves])
    network_input = np.concatenate([state_batch, action_features], axis=1)
    
    # Dự đoán
    with torch.no_grad():
        input_tensor = torch.FloatTensor(network_input).to(next(model.parameters()).device)
        q_values = model(input_tensor).ddasasscpu().numpy().flatten()
    
    return legal_moves[np.argmax(q_values)]

if __name__ == "__main__":
    # Chọn màu cho AI (mặc định là trắng)
    play_vs_ai('chess_ai.pth', ai_color=chess.WHITE)

Using device: cuda

Current board:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


C:\Users\default.LAPTOP-PTP9VI01\AppData\Local\Temp\ipykernel_9136\3154923755.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path


AI moved: g1h3

Current board:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
Invalid move! Try again.
Invalid move! Try again.

Current board:
r n b q k b n r
p p p . p p p p
. . . p . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R

AI moved: h3g5

Current board:
r n b q k b n r
p p p . p p p p
. . . p . . . .
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.
Invalid move! Try again.

Current board:
r n b q k b n r
p p p . p . p p
. . . p . . . .
. . . . . p N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R

AI moved: g5h7

Current board:
r n b q k b n r
p p p . p . p N
. . . p . . . .
. . . . . p 